# Schoonhoven
In this notebook we build a model for the area around Schoonhoven. We mainly focus on the surface water. There are three water boards in the model area, of which we download seasonal data about the stage of the surface water. For locations without a stage from the water board, we deliniate information from a Digital Terrain Model, to set a stage. Finally, for the river Lek, we build a river package with a fixed stage of 0.0 meter.
## Import packages

In [ ]:
import os
import rioxarray
import pandas as pd
from rasterstats import zonal_stats
from shapely.geometry import Point
import flopy
import logging
import matplotlib
import nlmod

# set the log-level to INFO, so more information is shown (compared to the default setting of WARNING)
logging.basicConfig(level=logging.INFO)

## Model settings
We define some model settings, like the name, the directory of the model files, the model extent and the time

In [ ]:
model_name = "Schoonhoven"
model_ws = os.path.join("models", model_name)
figdir, cachedir = nlmod.util.get_model_dirs(model_ws)
extent = [116500, 120000, 439000, 442000]
time = pd.date_range("2015", "2022", freq="MS") # monthly timestep

## Download data
### AHN
Download the Digital Terrain model of the Netherlands (AHN) with a resolution of 0.5 meter.

In [ ]:
fname_ahn = os.path.join(cachedir, "ahn.tif")
if not os.path.isfile(fname_ahn):
    ahn = nlmod.read.ahn.get_ahn4(extent, identifier='AHN4_DTM_05m')
    ahn.rio.to_raster(fname_ahn)

### layer 'waterdeel' from bgt
As the source of the location of the surface water bodies we use the 'waterdeel' layer of the Basisregistratie Grootschalige Topografie (BGT). This data consists of detailed polygons, maintained by dutch government agencies (water boards, municipalities and Rijkswatrstaat).

In [ ]:
bgt = nlmod.read.bgt.get_bgt(extent)

#### Add mimimum surface height around surface water bodies
Get the minimum surface level in 1 meter around surface water levels and add these data to the column 'ahn_min'.

In [ ]:
stats = zonal_stats(bgt.geometry.buffer(1.0), fname_ahn, stats="min")
bgt["ahn_min"] = [x["min"] for x in stats]

#### Plot 'bronhouder'
We can plot the column 'bronhouder' from the GeoDataFrame bgt. We see there are three water boards in this area (with codes starting with 'W').

In [ ]:
f, ax = nlmod.plot.get_map(extent)
bgt.plot("bronhouder", legend=True, ax=ax);

### level areas
For these three waterboards we download the level areas (peilgebieden): polygons with information about winter and summer stages.

In [ ]:
pg = nlmod.mfpackages.surface_water.download_level_areas(bgt, extent=extent)

#### Plot summer stage
The method download_level_areas() generates a dictionary with the name of the water boards as keys and GeoDataFrames as values. Each GeoDataFrame contains the columns summer_stage and winter_stage. Let's plot the summer stage, together with the location of the surface water bodies.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
bgt.plot(color="k", ax=ax)
for wb in pg:
    pg[wb].plot("summer_stage", ax=ax, vmin=-3, vmax=1, zorder=0)

#### Add stages to bgt-data
We then add the information from these level areas to the surface water bodies.

In [ ]:
bgt = nlmod.mfpackages.surface_water.add_stages_from_waterboards(bgt, pg=pg)

#### Plot summer stage of surface water bodies
We can plot the summer stage. There are some surface water bodies without a summer-stage, because the 'bronhouder' is not a water board. The main one is the river Lek, but there are also some surface water bodies without a summer stage more north. If no information about the stage is available, a constant stage is set to the minimal height of the digital terrain model (AHN) near the surface water body.

In [ ]:
f, ax = nlmod.plot.get_map(extent)
norm = matplotlib.colors.Normalize(vmin=-3, vmax=1)
cmap = 'viridis'
bgt.plot("summer_stage", ax=ax, norm=norm, cmap=cmap);
nlmod.plot.colorbar_inside(norm=norm, cmap=cmap);

There are locations without information about the stages from the water boards. That's why we also add the minimum surface level elevation around the surface water bodies, which is used as a backup. We can plot these values as well:

In [ ]:
f, ax = nlmod.plot.get_map(extent)
bgt.plot("ahn_min", ax=ax, norm=norm, cmap=cmap)
nlmod.plot.colorbar_inside(norm=norm, cmap=cmap);

### REGIS
For the schematisation of the subsurface we use REGIS. Let's download this data for the required extent.

In [ ]:
regis = nlmod.read.get_regis(extent, cachedir=cachedir, cachename="regis.nc")
regis

We then create a regular grid, add nessecary variables (eg idomain) and fill nan's. For example, REGIS does not contain infomration about the hydraulic conductivity of the first layer ('HLc'). These NaN's are replaced by a default hydraulic conductivity (kh) of 1 m/d. This probably is not a good representation of the conductivity, but at least the model will run.

In [ ]:
ds = nlmod.read.regis.to_model_ds(
    regis, model_name, model_ws, delr=100.0, delc=100.0
)
ds

## Add grid refinement
With the refine method, we can add grid refinement. The model will then use the disv-package instead of the dis-package. We can also test if the disv-package gives the same results as the dis-package by not specifying refinement_features: ds = nlmod.mgrid.refine(ds).

This notebook can be run with or without running the cell below.

In [ ]:
refinement_features = [(bgt[bgt["bronhouder"] == "L0002"], 2)]
ds = nlmod.mgrid.refine(ds, refinement_features=refinement_features)

## Add information about time

In [ ]:
ds = nlmod.mdims.set_model_ds_time(ds, time=time)

## Add knmi recharge to the model dataset

In [ ]:
knmi_ds = nlmod.read.knmi.get_recharge(
    ds, cachedir=cachedir, cachename="recharge.nc"
)
ds.update(knmi_ds)

## Create a groundwater flow model
Using the data from the xarray Dataset ds we generate a groundwater flow model.

In [ ]:
# %% create sim, tdis, gwf and ims
sim, gwf = nlmod.mfpackages.sim_tdis_gwf_ims_from_model_ds(ds)

# Create discretization
nlmod.mfpackages.dis_from_model_ds(ds, gwf)

# create node property flow
nlmod.mfpackages.npf_from_model_ds(ds, gwf)

# Create the initial conditions package
nlmod.mfpackages.ic_from_model_ds(ds, gwf, starting_head=0.0)

# Create the output control package
nlmod.mfpackages.oc_from_model_ds(ds, gwf)

# create recharge package
rch = nlmod.mfpackages.rch_from_model_ds(ds, gwf)

# create storagee package
sto = nlmod.mfpackages.sto_from_model_ds(ds, gwf)

## Process surface water
We cut the surface water bodies with the grid, set a default resistance of 1 day, and seperate the large river 'Lek' form the other surface water bodies.

In [ ]:
mg = nlmod.mgrid.modelgrid_from_model_ds(ds)
gi = flopy.utils.GridIntersect(mg, method="vertex")
bgt_grid = nlmod.mdims.gdf2grid(bgt, ix=gi).set_index("cellid")
bgt_grid["cond"] = bgt_grid.area / 1.0
mask = bgt_grid["bronhouder"] == "L0002"
lek = bgt_grid[mask]
bgt_grid = bgt_grid[~mask]

### Lek as river
Model the river Lek as a river with a fixed stage of 0.5 m NAP

In [ ]:
lek["stage"] = 0.0
lek["rbot"] = -3.0
spd = nlmod.mfpackages.surface_water.build_spd(lek, "RIV", ds)
riv = flopy.mf6.ModflowGwfriv(gwf, stress_period_data={0: spd})

### Other surface water as drains
model the other surface water using the drain package, with a summer stage and a  winter stage

In [ ]:
nlmod.mfpackages.surface_water.gdf_to_seasonal_pkg(bgt_grid, gwf, ds);

## Run the model

In [ ]:
nlmod.util.write_and_run_model(gwf, ds)

## Post-processing
### Get the simulated head

In [ ]:
head = nlmod.util.get_heads_dataarray(ds)

### Plot the average head

In [ ]:
f, ax = nlmod.plot.get_map(extent)
pc = nlmod.plot.da(head.sel(layer="HLc").mean("time"), ds=ds, edgecolor="k")
nlmod.plot.colorbar_inside(pc)
bgt.plot(ax=ax, edgecolor="k", facecolor="none");

### plot a time series at a certain location

In [ ]:
x = 118228
y = 439870
if ds.gridtype == "vertex":
    icelld2 = gi.intersect(Point(x, y))["cellids"][0]
    head_point = head[:, :, icelld2]
else:
    head_point = head.interp(x=x, y=y, method="nearest")
head_point.plot.line(hue="layer", size=(10));

### Plot some properties of the first layer

In [ ]:
layer = "HLc"
f, axes = nlmod.plot.get_map(extent, nrows=2, ncols=2)
nlmod.plot.da(ds["idomain"].sel(layer=layer), ds=ds, ax=axes[0, 0])
nlmod.plot.da(ds["kh"].sel(layer=layer), ds=ds, ax=axes[1, 0])
nlmod.plot.da(ds["top"], ds=ds, ax=axes[0, 1])
nlmod.plot.da(ds["botm"].sel(layer=layer), ds=ds, ax=axes[1, 1])